In [1]:
import sys
sys.path.append('/lmh_data/work/sclab/train/')

import numpy as np
import pandas as pd
import torch
import torch.utils.data as data
from torch.autograd import Variable
from captum.attr import IntegratedGradients

from dataset import Dataset
from net import define_network

In [2]:
torch.manual_seed(123)
np.random.seed(123)

In [3]:
def evaluate(eval_file, model_file, gene_name):
    eval_set = Dataset(eval_file, gene_name, is_train=True)
    data_loader = data.DataLoader(eval_set, batch_size=1, shuffle=False)

    input_size, output_size = tuple(eval_set[0][0].shape), eval_set[0][1].shape[0]
    patch_size = tuple([int(i / 8) for i in input_size])
    model = torch.nn.DataParallel(define_network(input_size, patch_size, output_size))
    model.load_state_dict(torch.load(model_file))
    model.cuda()
    model.eval()
    ig = IntegratedGradients(model)

    _datas = np.load(eval_file, allow_pickle=True)
    important_genes = dict(Astro=dict(), Endo=dict(), OPC=dict(), ODC=dict())
    for index, batch in enumerate(data_loader, 1):
        if _datas[index-1]['cell_type'] != 'ODC':
            continue
        _important_genes = important_genes[_datas[index-1]['cell_type']]
        input = Variable(batch[0]).cuda().unsqueeze(1)
        target = Variable(batch[1]).cuda().unsqueeze(1) / 10
        
        attributions = torch.zeros(input.reshape(-1).shape).cuda()
        for i in range(target.shape[-1]):
            attributions += ig.attribute(input, target=(0, i))[0, 0].reshape(-1)
        attributions = attributions.cpu().detach().numpy()
        for i in range(len(attributions)):
            if i not in _important_genes.keys():
                _important_genes[i] = 0
            _important_genes[i] += attributions[i]
        
    return important_genes

important_genes = evaluate(
    '/lmh_data/data/sclab/sclab/promoter/eval_dataset.npy',
    '/lmh_data/data/sclab/sclab/promoter/tmp/QKI/model_epoch_9.pth', 'QKI')
important_genes

{'Astro': {},
 'Endo': {},
 'OPC': {},
 'ODC': {0: 0.0,
  1: 0.0,
  2: 0.0,
  3: 0.0,
  4: 0.0,
  5: 0.0,
  6: 0.0,
  7: 0.0,
  8: 0.0,
  9: 0.0,
  10: 0.0,
  11: 0.0,
  12: 0.0,
  13: 0.0,
  14: 0.0,
  15: 0.0021148147206986323,
  16: 0.0,
  17: 0.0,
  18: 0.0,
  19: -0.05605316627770662,
  20: -0.0185400964692235,
  21: 0.0,
  22: -0.02843006420880556,
  23: 0.0,
  24: 0.003262626909872779,
  25: 0.0,
  26: 0.2173838319722563,
  27: 0.0,
  28: 0.0,
  29: 0.0,
  30: 0.0,
  31: 0.0,
  32: 0.0,
  33: 0.0,
  34: -0.036361354985274374,
  35: -0.012299590278416872,
  36: 0.0,
  37: -0.10291233286261559,
  38: -0.02628622157499194,
  39: 0.08634522068314254,
  40: 0.03567761415615678,
  41: 0.21812419965863228,
  42: 0.0,
  43: -0.08397234650328755,
  44: 0.0,
  45: -0.23649329552426934,
  46: -0.16695789014920592,
  47: -0.15107628726400435,
  48: -0.011251431722484995,
  49: -0.015832956414669752,
  50: 0.06538181391078979,
  51: -0.04907453409396112,
  52: 0.03383023617789149,
  53: 0.06

In [4]:
scRNA_head = np.load('/lmh_data/data/sclab/sclab/eval_dataset.npy', allow_pickle=True)[0]['scRNA_head']

def find_special_genes(important_genes, cell_type):
    genes = pd.DataFrame(
        sorted(important_genes[cell_type].items(), key=lambda x: x[1], reverse=True), columns=['index', 'value']
    ).set_index('index')
    genes = genes.sort_values(by=['value'], ascending=False)
    return genes

genes = find_special_genes(important_genes, 'ODC')
genes['name'] = scRNA_head[list(genes.index)]
genes.to_csv('tmp/related_genes_QKI_promoter.csv')
genes

,value,name
index,,
18428,192.080493,PCDH9
15663,93.069224,FTH1
11895,82.806347,ST18
28301,75.318168,PLP1
28113,65.286977,XIST
...,...,...
6516,-29.921457,ATP8A1
726,-30.572371,RNF220
3848,-38.833105,FMNL2
